# TLdetect

An implementation of a convolutional neural network to detect the state of traffic lights in images. The states are red, green, yellow and no traffic light. Using the **tensorflow** framework.

### Importing packages

In [1]:
from PIL import Image
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import src.data_processing as data

## Defining global parameters

In [2]:
NUM_ITERATIONS = 100
BATCH_SIZE = 64
KERNEL_SIZE = (3, 3)

### Pre-processing

Loading the images and labels and treating the dataset to fit better on our model. Note that the images won't be loaded right now, only their file names, in order to save memory and avoid bottlenecks and errors.

In [3]:
# Loading the labels
# At this moment, the conv net will only classify if an image has or not a traffic light.
labels = data.load_labels()
print("Number of labels loaded: %d" % len(labels))

Number of labels loaded: 13063


In [4]:
# Loading the images. In reality, just their names
images = labels['file']
X_train, X_val, X_test = data.split_dataset(images)

# Debug
print("Data set of %d images split in 3 sets.\n" % len(images))
print("Train: {}".format(X_train.shape))
print("Validation: {}".format(X_val.shape))
print("Test: {}".format(X_test.shape))

Data set of 13063 images split in 3 sets.

Train: (9438,)
Validation: (1665,)
Test: (1960,)


In [5]:
# Spliting the labels
y_train = data.extract_labels(labels, X_train)
y_val = data.extract_labels(labels, X_val)
y_test = data.extract_labels(labels, X_test)

# Debug
print("Lables extracted.\n")
print("Train labels shape: {}".format(y_train.shape))
print("Train validation shape: {}".format(y_val.shape))
print("Train test shape: {}".format(y_test.shape))

Lables extracted.

Train labels shape: (9438,)
Train validation shape: (1665,)
Train test shape: (1960,)


### Creating the model

At the following cells we will define our model. Our model, at this moment, will just classify if a given image has a traffic light or not.

Note that the inputs pass through two max-pooling layers before starts to be recognized. This technique is used to reduce the dimensionality of the inputs.

The original inputs has the dimension of (1200, 1920, 3).

#### The following architecture will be used:
1. **Pooling layer**
+ **Pooling layer** (Output dimension will be (300, 480, 3))
+ **Convolutional layer**
+ **Pooling layer**
+ **Convolutional layer**
+ **Pooling layer**
+ **Affine layer**
+ **Classification**

## Defining the architecture

In [6]:
from tensorflow.contrib.keras import layers, models, backend

model = models.Sequential()
model.add(layers.MaxPool2D(pool_size=2, strides=2,
                           data_format='channels_last',
                           input_shape=(1200, 1920, 3)
                          ))
model.add(layers.MaxPool2D(pool_size=2, strides=2))
model.add(layers.MaxPool2D(pool_size=2, strides=2))

model.add(layers.Conv2D(filters=32, kernel_size=KERNEL_SIZE, strides=1,
                        padding='same',
                        activation='relu'
                       ))
model.add(layers.MaxPool2D(pool_size=2, strides=2))
model.add(layers.Conv2D(filters=32, kernel_size=KERNEL_SIZE, strides=1,
                        padding='same',
                        activation='relu'
                       ))
model.add(layers.Conv2D(filters=32, kernel_size=KERNEL_SIZE, strides=1,
                        padding='same',
                        activation='relu'
                       ))
model.add(layers.MaxPool2D(pool_size=2, strides=2))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# CIFAR-10 Test

In [ ]:
from tensorflow.contrib.keras import datasets
from tensorflow.contrib.keras import layers, models, backend

(imgs_cif, imgs_labels), (_, _) = datasets.cifar10.load_data()
imgs_labels[0]

In [ ]:
from tensorflow.contrib.keras import utils, optimizers

imgs_labels = utils.to_categorical(imgs_labels, 10)
print("Data shape: {}, and labels: {}".format(imgs_cif[0].shape, imgs_labels.shape))
print(imgs_labels[0])

# Architecture
model_cif = models.Sequential()
model_cif.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=imgs_cif.shape[1:]))
model_cif.add(layers.Activation('relu'))
model_cif.add(layers.Conv2D(32, (3, 3)))
model_cif.add(layers.Activation('relu'))
model_cif.add(layers.MaxPool2D(pool_size=2))

model_cif.add(layers.Conv2D(64, (3, 3), padding='same'))
model_cif.add(layers.Activation('relu'))
model_cif.add(layers.Conv2D(64, (3, 3), padding='same'))
model_cif.add(layers.Activation('relu'))
model_cif.add(layers.MaxPool2D(pool_size=2))

model_cif.add(layers.Flatten())
model_cif.add(layers.Dense(512, activation='relu'))
model_cif.add(layers.Dense(10, activation='softmax'))

opt = optimizers.RMSprop(lr=0.0001, decay=1e-6)

model_cif.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_cif.fit(imgs_cif, imgs_labels, batch_size=BATCH_SIZE, epochs=NUM_ITERATIONS, shuffle=True)

# ----------------------------------------------

## Training the model

In [7]:
report = model.fit_generator(data.images_generator(X_train[:BATCH_SIZE*4], y_train[:BATCH_SIZE*4], BATCH_SIZE),
#                    int(X_train.shape[0]/BATCH_SIZE),
                    4,
                    epochs=NUM_ITERATIONS
                   )

Epoch 1/100
3/4 [=====================>........] - ETA: 6s - loss: 10.1859 - acc: 0.3646 

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/contrib/keras/python/keras/engine/training.py", line 615, in data_generator_task
    generator_output = next(self._generator)
StopIteration



ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None